In [1]:
## import general libraries
import numpy as np
import pandas as pd
import re
import json
import glob
# import nltk
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
# from gensim.models import coherencemodel
from APIs.Data_Cleaning import datacleaning, datamapping, datapreprocessing

import nltk
nltk.data.path.append("/home/fibebocai/nltk_data/")
from nltk.corpus import stopwords
import en_core_web_sm
from nltk.stem import WordNetLemmatizer

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
import ray

2022-06-23 07:59:16.838000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/lsf_10.1.0/10.1/linux3.10-glibc2.17-x86_64/lib:/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pyoracleclient/instantclient
2022-06-23 07:59:16.838020: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from APIs.Data_Processing import Data_Processing
ticketname = "Data/tickets (6).xlsx"
stopwords = stopwords.words("english")

"""if you find some words which is causing noise in the topic creation or which are not contributing much in the topic 
generation then add the word below"""

stopwords.extend(['from', 'subject', 're', 'edu', 'use', 'infineon', 'camstar','com', 'screenshot','yoda','lot','iptester',\
                        "mkz","mkzsapa","mkzsapaxx","site","xml", "total","camstarportal","camstarweb","team","go","gk","gl","alf",\
                        "give","zhouyang","morning","get","getquerytext","ghani","unknown","glue","generate","generation","green",\
                        "afternoon","good","evening","night","user","freeze","full","forward","hello","baby","id", "cannot","unable","add",\
                        "flex", "error message", "error","able","due","mgt","sba","allow","bin","still","workflow","hence","ok","since","critical",\
                        "urgent", "priority", "request","need", "main", "create","would","like","track","must",\
                        "prior","high","find","type","select", "job","fa","type","show", "status",\
                        "move","print"])
a = Data_Processing(ticketname, stopwords) 
[df_reviews, data_words] = a.getdata()

/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
import ClusterTransformer.ClusterTransformer as ctrans
import pandas as pd

cr = ctrans.ClusterTransformer()
model_name = "allenai/longformer-base-4096"

In [4]:
batch_size = 2
max_seq_length = 64
convert_to_numpy = False
normalize_embeddings = False
neighborhood_min_size = 1
cutoff_threshold = 0.83
kmeans_max_iter = 100
kmeans_random_state = 42
kmeans_no_cluster = 6
li_sentence = df_reviews['Desc']

In [ ]:
ray.shutdown()
ray.init(num_cpus=16)
@ray.remote
def train(batch_size, li_sentence, model_name, max_seq_length, convert_to_numpy, normalize_embeddings, neighborhood_min_size, cutoff_threshold, kmeans_max_iter, kmeans_random_state, kmeans_no_cluster):
    embeddings = cr.model_inference(li_sentence, batch_size, model_name, max_seq_length, normalize_embeddings, convert_to_numpy)
    output_dict = cr.neighborhood_detection(li_sentence, embeddings, cutoff_threshold, neighborhood_min_size)
    output_kmeans_dict = cr.kmeans_detection(li_sentence, embeddings, kmeans_no_cluster, kmeans_max_iter, kmeans_random_state)
    neighborhood_detection_df = cr.convert_to_df(output_dict)
    kmeans_df = cr.convert_to_df(output_kmeans_dict)
    return [neighborhood_detection_df, kmeans_df, output_kmeans_dict, output_dict, embeddings]

[neighborhood_detection_df, kmeans_df, output_kmeans_dict, output_dict, embeddings] = ray.get(train.remote(batch_size, li_sentence, model_name, max_seq_length, convert_to_numpy, normalize_embeddings, neighborhood_min_size, cutoff_threshold, kmeans_max_iter, kmeans_random_state, kmeans_no_cluster))


File descriptor limit 4000 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'
2022-06-23 07:59:40,221	INFO services.py:1476 -- View the Ray dashboard at http://127.0.0.1:8265
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fr

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

...

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
(train pid=3946818) - This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(train pid=3946818) - This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(train pid=3946818) Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.

In [ ]:
print(f'DataFrame from neighborhood detection:\n {neighborhood_detection_df}')
print(f'DataFrame from Kmeans detection:\n {kmeans_df}')
cr.plot_cluster(neighborhood_detection_df)
cr.plot_cluster(kmeans_df)